# Preprocessing the Data

In [96]:
import boto3
import os

# Define S3 bucket and file names
s3_bucket = "fake-news-raw-data"
s3_files = ["Liar.csv", "Synthetic Financial Datasets.csv", "WELFake_Dataset.csv"]
local_folder = "/home/ec2-user/SageMaker/data/"

# Ensure local directory exists
os.makedirs(local_folder, exist_ok=True)

# Initialize S3 client
s3_client = boto3.client("s3")

# Download files from S3
for file in s3_files:
    local_path = os.path.join(local_folder, file)
    s3_client.download_file(s3_bucket, file, local_path)
    print(f"✅ Downloaded {file} to {local_path}")

✅ Downloaded Liar.csv to /home/ec2-user/SageMaker/data/Liar.csv
✅ Downloaded Synthetic Financial Datasets.csv to /home/ec2-user/SageMaker/data/Synthetic Financial Datasets.csv
✅ Downloaded WELFake_Dataset.csv to /home/ec2-user/SageMaker/data/WELFake_Dataset.csv


## Import Python package "stopwords" to overlook commonly used words and articles (English)

In [97]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Import required modules

In [98]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

## Ensure NLTK stopwords are available

In [99]:
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load Liar.csv dataset and make pandas dataframe

In [100]:
liar_clean = pd.read_csv("/home/ec2-user/SageMaker/data/Liar.csv")

# Display dataset info

liar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Statement ID          10240 non-null  object 
 1   Lie_label             10240 non-null  object 
 2   Statement             10240 non-null  object 
 3   Topic                 10238 non-null  object 
 4   Speaker               10238 non-null  object 
 5   Speaker_Job_Title     7342 non-null   object 
 6   State                 8030 non-null   object 
 7   Speaker_party         10238 non-null  object 
 8   barely_true_counts    10238 non-null  float64
 9   false_counts          10238 non-null  float64
 10  half-true_counts      10238 non-null  float64
 11  mostly_true_counts    10238 non-null  float64
 12  pants_on_fire_counts  10238 non-null  float64
 13  statement_mode        10138 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.1+ MB


## Function to clean text using NLTK stopwords

In [101]:
def clean_text_nltk(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

liar_clean["clean_statement"] = liar_clean["Statement"].apply(clean_text_nltk)

## Convert numerical columns to integers

In [102]:
count_cols = ["barely_true_counts", "false_counts", "half-true_counts", "mostly_true_counts", "pants_on_fire_counts"]
liar_clean[count_cols] = liar_clean[count_cols].fillna(0).astype(int)

## Create a total misinformation score feature

In [103]:
liar_clean["total_misinfo_score"] = liar_clean[count_cols].sum(axis=1)

## One-Hot Encoding for Speaker Party

In [104]:
encoder = OneHotEncoder(sparse_output=False,drop="first")
encoded_party = encoder.fit_transform(liar_clean[["Speaker_party"]])
party_columns = encoder.get_feature_names_out(["Speaker_party"])
liar_encoded_party = pd.DataFrame(encoded_party, columns=party_columns, index=liar_clean.index)

## Merge and drop original categorical column

In [105]:
liar_clean = pd.concat([liar_clean, liar_encoded_party], axis=1)
liar_clean.drop(["Speaker_party"], axis=1, inplace=True)

## Balance dataset by oversampling minority classes

In [106]:
majority_class = liar_clean[liar_clean["Lie_label"] == "FALSE"]
minority_classes = liar_clean[liar_clean["Lie_label"] != "FALSE"]
minority_classes_upsampled = resample(minority_classes, replace=True, n_samples=len(majority_class), random_state=42)
liar_balanced = pd.concat([majority_class, minority_classes_upsampled])

## Split dataset

In [107]:
train_data, test_data = train_test_split(liar_balanced, test_size=0.2, random_state=42, stratify=liar_balanced["Lie_label"])

## Save the preprocessed data

In [108]:
train_data.to_csv("/home/ec2-user/SageMaker/data/Liar_train.csv", index=False)
test_data.to_csv("/home/ec2-user/SageMaker/data/Liar_test.csv", index=False)

print("Preprocessing on Liar dataset complete. Training and test datasets saved.")


Preprocessing on Liar dataset complete. Training and test datasets saved.


**References**

OpenAI. (2025). ChatGPT (March 20 version). [LLM]. https://chatgpt.com

Python Tutorials. (2021, July 22). *NLTK stop words.* pythonspot. Accessed March 20, 2025 from https://pythonspot.com/nltk-stop-words/